In [1]:
import os

for root, dirs, files in os.walk(".", topdown=True):
    for f in files:
        if f.endswith(".dat"):
            print(os.path.join(root, f))


In [3]:
import pandas as pd

ratings = pd.read_csv(
    r"C:\Users\yagne\OneDrive\Desktop\JOB\projects\Recommender-System-with-Neural-Networks\data\raw\ml-1m\ratings.dat",
    sep="::",
    engine="python",
    names=["user_id", "item_id", "rating", "timestamp"]
)

users = pd.read_csv(
    r"C:\Users\yagne\OneDrive\Desktop\JOB\projects\Recommender-System-with-Neural-Networks\data\raw\ml-1m\users.dat",
    sep="::",
    engine="python",
    names=["user_id", "gender", "age", "occupation", "zip"]
)

movies = pd.read_csv(
    r"C:\Users\yagne\OneDrive\Desktop\JOB\projects\Recommender-System-with-Neural-Networks\data\raw\ml-1m\movies.dat",
    sep="::",
    engine="python",
    encoding="latin-1",
    names=["item_id", "title", "genres"]
)

csv_path = r"C:\Users\yagne\OneDrive\Desktop\JOB\projects\Recommender-System-with-Neural-Networks\data\raw\csv_files"

ratings.to_csv(f"{csv_path}\\ratings.csv", index=False)
users.to_csv(f"{csv_path}\\users.csv", index=False)
movies.to_csv(f"{csv_path}\\movies.csv", index=False)




In [4]:

movies.head()

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings_df = pd.read_csv(r"C:\Users\yagne\OneDrive\Desktop\JOB\projects\Recommender-System-with-Neural-Networks\data\raw\csv_files\ratings.csv")
movies_df = pd.read_csv(r"C:\Users\yagne\OneDrive\Desktop\JOB\projects\Recommender-System-with-Neural-Networks\data\raw\csv_files\movies.csv")
users_df = pd.read_csv(r"C:\Users\yagne\OneDrive\Desktop\JOB\projects\Recommender-System-with-Neural-Networks\data\raw\csv_files\users.csv")

## Cleaning ratings csv

In [3]:
c_ratings_df = ratings_df.copy()
c_ratings_df.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
c_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1000209 non-null  int64
 1   item_id    1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [5]:
c_ratings_df.describe()

,user_id,item_id,rating,timestamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


Checking for nulls

In [6]:
c_ratings_df.isnull().sum()


user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

Converting timestamp

In [7]:
c_ratings_df['timestamp'] = pd.to_datetime(c_ratings_df['timestamp'], unit='s')

Remove duplicates

In [8]:
c_ratings_df = c_ratings_df.drop_duplicates()


Checking ranges

In [9]:
c_ratings_df['rating'].unique()

array([5, 3, 4, 2, 1])

In [10]:
c_ratings_df.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


## Cleaning users 

In [11]:
c_users_df = users_df.copy()
c_users_df.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [12]:
c_users_df.describe()

,user_id,age,occupation
count,6040.000000,6040.000000,6040.000000
mean,3020.500000,30.639238,8.146854
std,1743.742145,12.895962,6.329511
min,1.000000,1.000000,0.000000
25%,1510.750000,25.000000,3.000000
50%,3020.500000,25.000000,7.000000
75%,4530.250000,35.000000,14.000000
max,6040.000000,56.000000,20.000000


Converting gender to numeric

In [13]:
c_users_df['gender'] = c_users_df['gender'].map({'M': 1, 'F': 0})

In [14]:
c_users_df.head()

,user_id,gender,age,occupation,zip
0,1,0,1,10,48067
1,2,1,56,16,70072
2,3,1,25,15,55117
3,4,1,45,7,02460
4,5,1,25,20,55455


In [15]:
c_users_df = c_users_df.drop(columns = ["zip"])

Converting occupation to categories

In [16]:
c_users_df['occupation'] = c_users_df['occupation'].astype("category").cat.codes

In [17]:
c_users_df.dtypes

user_id       int64
gender        int64
age           int64
occupation     int8
dtype: object

In [18]:
c_users_df["occupation"].unique()

array([10, 16, 15,  7, 20,  9,  1, 12, 17,  0,  3, 14,  4, 11,  8, 19,  2,
       18,  5, 13,  6], dtype=int8)

In [19]:
# cat = c_users_df['occupation'].astype("category")
# c_users_df['occupation_cat'] = cat
# c_users_df['occupation_code'] = cat.cat.codes


In [20]:
c_users_df.isnull().sum()


user_id       0
gender        0
age           0
occupation    0
dtype: int64

In [21]:
unique_users = sorted(c_users_df['user_id'].unique())
user_map = {u: i for i, u in enumerate(unique_users)}

c_users_df['user_id'] = c_users_df['user_id'].map(user_map)


c_users_df = c_users_df.astype({
    "user_id": "int32",
    "gender": "int8",
    "age": "int8",
    "occupation": "int8"
})

In [22]:
c_users_df.head()


,user_id,gender,age,occupation
0,0,0,1,10
1,1,1,56,16
2,2,1,25,15
3,3,1,45,7
4,4,1,25,20


## Cleaning Movies 

In [23]:
c_movies_df = movies_df.copy()
c_movies_df.head()

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [31]:
c_movies_df = c_movies_df.rename(columns={'item_id': 'movie_id'})


In [33]:
c_movies_df.columns

Index(['movie_id', 'title', 'genres'], dtype='object')

In [34]:
c_movies_df.describe()

,movie_id
count,3883.000000
mean,1986.049446
std,1146.778349
min,1.000000
25%,982.500000
50%,2010.000000
75%,2980.500000
max,3952.000000


In [35]:
c_movies_df.isnull().sum()

movie_id    0
title       0
genres      0
dtype: int64

In [36]:
c_movies_df.isna().sum()

movie_id    0
title       0
genres      0
dtype: int64

In [37]:
unique_movies = sorted(c_movies_df['movie_id'].unique())
movie_map = {m: i for i, m in enumerate(unique_movies)}

c_movies_df['movie_id'] = c_movies_df['movie_id'].map(movie_map)


In [38]:
c_movies_df.head()

,movie_id,title,genres
0,0,Toy Story (1995),Animation|Children's|Comedy
1,1,Jumanji (1995),Adventure|Children's|Fantasy
2,2,Grumpier Old Men (1995),Comedy|Romance
3,3,Waiting to Exhale (1995),Comedy|Drama
4,4,Father of the Bride Part II (1995),Comedy


In [39]:
c_movies_df['year'] = c_movies_df['title'].str.extract(r'\((\d{4})\)')
c_movies_df['year'] = c_movies_df['year'].fillna(0).astype('int16')


In [40]:
c_movies_df['clean_title'] = c_movies_df['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()


In [41]:
genres_split = c_movies_df['genres'].str.get_dummies(sep='|')
c_movies_df = pd.concat([c_movies_df, genres_split], axis=1)


In [45]:
c_movies_df.head()

,movie_id,title,genres,year,clean_title,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Jumanji (1995),Adventure|Children's|Fantasy,1995,Jumanji,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,2,Grumpier Old Men (1995),Comedy|Romance,1995,Grumpier Old Men,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,3,Waiting to Exhale (1995),Comedy|Drama,1995,Waiting to Exhale,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Saving cleaned data 

In [47]:
c_users_df.to_csv(r"C:\Users\yagne\OneDrive\Desktop\JOB\projects\Recommender-System-with-Neural-Networks\data\processed\users_clean.csv", index=False)
c_movies_df.to_csv(r"C:\Users\yagne\OneDrive\Desktop\JOB\projects\Recommender-System-with-Neural-Networks\data\processed\movies_clean.csv", index=False)
ratings_df.to_csv(r"C:\Users\yagne\OneDrive\Desktop\JOB\projects\Recommender-System-with-Neural-Networks\data\processed\ratings_clean.csv", index=False)
